In [1]:
import sklearn
import sys
# sys.path.insert(1, "../../sklearn-porter/sklearn_porter")
sys.path.insert(1, "../../sklearn-porter")
# from sklearn_porter import Porter
# import Porter
import pickle
from sklearn.externals import joblib
import joblib
import numpy as np
import pandas as pd
import os
import re
import json

import ccx as cx
# import sklearn_porter.Porter
from sklearn_porter import Porter

from sklearn.tree import export_graphviz

In [ ]:
# Porter.__version__
Porter.

In [ ]:
# specifically needs version 0.20.2
# can not open pickle file for later versions
# activate the virtual environment venv
sklearn.__version__

In [9]:
MODEL_PATH = '../model/'
OUT_DIR = '../website/js/extraTrees/'
model_2_3_path = MODEL_PATH + 'model_2_3.pkl'
# 3 to 2 is used
model_3_2_path = MODEL_PATH + 'model_3_2.pkl'

In [10]:
model_2_3 = joblib.load(model_2_3_path)
model_3_2 = joblib.load(model_3_2_path)

In [11]:
print(len(model_2_3))
print(len(model_3_2))

198
242


In [12]:
random_input = np.random.randint(0,6,211).reshape(1,-1).astype(float)
print(model_2_3[0].predict(random_input))
print(model_2_3[0].predict_proba(random_input))

[0]
[[0.69681154 0.30318846]]


In [16]:
porter = Porter(model_3_2[0], language='js')
output = porter.export()
with open(OUT_DIR + 'extraTrees.js', 'w') as f:
    f.write(output)

In [17]:
random_test_ints = np.random.randint(0,6,211 * 200).reshape(200,-1)
random_test_floats = random_test_ints.astype(float)
print(porter.integrity_score(random_test_ints))

ValueError: invalid literal for int() with base 10: '[ 0.6595365418894831, 0.3404634581105169 ]'

In [ ]:
model = model_3_2[0]
def drawTrees(model,PREF='../'):
    '''
        draw the estimators (trees)
        in a single model
    '''
    N=len(model.estimators_)

    for count in range(N):
        estimator = model.estimators_[count]

        export_graphviz(estimator, out_file=PREF+'tree.dot', 
                        #feature_names = iris.feature_names,
                        #class_names = iris.target_names,
                        rounded = True, proportion = False, 
                        precision = 2, filled = True)

        from subprocess import call
        call(['dot', '-Tpng',  PREF+'tree.dot', '-o', PREF+'tree'+str(count)+'.png', '-Gdpi=600'])
        from IPython.display import Image
        Image(filename = PREF+'tree'+str(count)+'.png') 
        
        
drawTrees(porter.estimator, "graphs/")

In [ ]:
# NOT NEEEDED
def get_question_numbers(model):
    """
    Args:
        extra trees model
        
    Returns:
        list of question numbers
    """
    
    nodeid=np.zeros(model.n_estimators).astype(int)
    responses={}
    while not all(nodeid<0):
        [nodeid,responses]=cx.getNodeid(model,nodeid=nodeid,responses=responses,ask=cx.ask)
    return list(responses.keys())

In [20]:
def extra_tree_to_js(model, out_file):
    """Create a js file of the extra trees.
    
    Args:
        model: extraTrees model
        out_file: file name to write to
        
    Returns:
        Porter
    """
    
    porter = Porter(model, language='js')
    output = porter.export()
    
    with open(out_file, 'w') as f:
        f.write(output)
        
    return porter
        
# write_sklearn_porter(model, OUT_DIR + 'extraTrees.js')

In [21]:
def extra_trees_to_js(models, out_dir):
    """Create multiple js files from list of extra trees
    
    Args:
        models: list of extra trees models
        out_dir: directory to write js files
        
    Returns:
        list of porters
    """
    
    porters = []
    for i, model in enumerate(models):
        name = str(i) + '.js'
        porter = extra_tree_to_js(model, os.path.join (out_dir, name))
        porters.append(porter)
    return porters
        
porters = extra_trees_to_js(model_3_2, OUT_DIR)

In [ ]:
def calc_integrity(porters, num_samples):
    """Calculate the integrity of the porter.
    
    Meaning it calculates the accuracy of the ported 
    classifier.
    The value is between 0 and 1.
    
    Args:
        porters: list of porters
        num_samples: number of samples for each porter to test
        
    Returns:
        average probability
    """
    
    scores = []
    for porter in porters:
        random_samples = np.random.randint(0,6,211 * num_samples).reshape(num_samples,-1)
        score = porter.integrity_score(random_samples)
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

calc_integrity(porters, 10)

In [22]:
def parse_extra_tree_js(file_name):
    """Parse the javascript file for the splits
    
    Args:
        file_name: name of file
        
    Returns:
        dictionary containing question numbers and threshold
    """
    
    split_points = {}
    q_numbers = []
    thresholds = []
    
    with open(file_name, 'r') as f:
        content = f.read()
        
    instances = re.findall(r'features\[\d+\] <= \d+\.\d+', content)
    
    for instance in instances:
        q_number = int(re.search(r'\d+|$', instance).group())
        threshold = float(re.search(r'\d+\.\d+', instance).group())
        threshold = round(threshold, 3)
        q_numbers.append(q_number)
        thresholds.append(threshold)
        
    split_points['q_numbers'] = q_numbers
    split_points['thresholds'] = thresholds 
    return split_points

split_dict = parse_extra_tree_js(OUT_DIR + 'extraTrees.js')
split_dict

{'q_numbers': [131, 143, 140, 3, 191, 82, 98, 152, 194, 69, 208, 170, 49, 129],
 'thresholds': [2.966,
  3.174,
  1.909,
  3.404,
  2.533,
  1.421,
  3.034,
  2.114,
  1.047,
  2.736,
  4.112,
  2.029,
  3.257,
  3.349]}

In [23]:
def parse_multiple_extra_tree_js(file_dir):
    """Parse multiple javascript files for splits.
    CAUTION: right now, we only take the files with exactly 14 splits.
    
    Args:
        file_dir: directory containing the javascript files
        
    Returns:
        
    """
    
    file_name_to_split_points = {}
    js_files = []
    #q_numbers = []
    
    for file in os.listdir(file_dir):
        if file.endswith(".js"):
            js_files.append(file)
    
    for js_file in js_files:
        split_points = parse_extra_tree_js(os.path.join(file_dir, js_file))
        
        if len(split_points['q_numbers']) == 14:
            file_name_to_split_points['f' + js_file.split('.')[0]] = split_points
        #q_numbers.append(split_points['q_numbers'])
    
    return file_name_to_split_points

split_dict = parse_multiple_extra_tree_js(OUT_DIR)

In [24]:
with open('../website/js/tree_splits.json', 'w') as json_file:
#     json.dump(split_dict, json_file, indent=4, sort_keys=True)
    json.dump(split_dict, json_file, sort_keys=True)

In [63]:
items = "ptsd_items.xlsx"
ptsd_items = pd.read_excel(items)
ptsd_items.set_index("Variable / Field Name", inplace=True)
ptsd_items.columns = ['questions', 'choices']
ptsd_items["choices"] = ptsd_items["choices"].map(lambda x: x.replace('|', '<br>').replace(",", ":"))

In [64]:
ptsd_items.head()

,questions,choices
Variable / Field Name,,
ptsd1,"During a stressful event in the past, how much...",1: Not at all <br> 2: A little bit <br> 3: Mod...
ptsd2,"During a stressful event in the past, how many...",1: None <br> 2: A few <br> 3: At least half <b...
ptsd3,"During a stressful event in the past, I person...",1: Not at all <br> 2: A little bit <br> 3: Mod...
ptsd4,"During a stressful event in the past, I observ...",1: Never <br> 2: Rarely <br> 3: Sometimes <br>...
ptsd5,"During a stressful event in the past, someone ...",1: Never <br> 2: Rarely <br> 3: Sometimes <br>...


In [65]:
ptsd_items["choices"].unique()

array(['1: Not at all <br> 2: A little bit <br> 3: Moderately <br> 4: Quite a bit <br> 5: Very much <br> ',
       '1: None <br> 2: A few <br> 3: At least half <br> 4: More than half <br> ',
       '1: Never <br> 2: Rarely <br> 3: Sometimes <br> 4: Often <br> '],
      dtype=object)

In [66]:
def ptsd_items_to_json():
    dict_ = {}
    for i, q in enumerate(ptsd_items.index):
        items = {}
        items["questions"] = ptsd_items["questions"][i]
        items["choices"] = ptsd_items["choices"][i]
        dict_[q] = items
#         print (q)
    return dict_
        
ptsd_items_dict = ptsd_items_to_json()


In [67]:
with open("../website/js/questions.json", 'w') as outfile:
    json.dump(ptsd_items_dict, outfile)

In [60]:
# for i in ptsd_items.index:
#     print(i)